**Transformación de datos con Pandas**

El siguiente proyecto, tiene como objetivo unificar varios archivos, almacenados en un directorio, con el fin de hacer un solo archivo unificado y, luego de eso, limpiar y transformar los datos, hasta tener una base final limpia y bien estructurada. Este es un proyecto real, realizado para una empresa de telecomunicaciones. Sin embargo, para la práctica, se usará una cantidad de archivos reducido, con data simplificada y con información ficticia, con el objetivo de proteger los datos privados de la empresa.

**Parte I: Importación de librerías y carga de datos al dataframe**

Antes que nada, nos conectaremos a nuestro Google Colab para extraer los archivos.

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 11.0 MB/s eta 0:00:00


Ahora importaremos las librerías necesarias y cargaremos el archivo a un df, utilizando la librería os, para crear listas de directorios y acceder a todos los archivos, para unificarlos.

In [25]:
import pandas as pd
import os
import glob

In [26]:
# Crear lista con los archivos presentes en el directorio y asignarlos a la variable "files"
files = os.listdir('/content/drive/My Drive/python_projects/XLS')

In [27]:
# Acceder a cada archivo con el ciclo 'for' y agregarlo a un dataframe
for i in files:
  df = pd.read_excel('/content/drive/My Drive/python_projects/XLS/'+i)

Realizaremos una exploración inicial de los datos

In [28]:
  print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID_ORDEN                    46 non-null     object 
 1   ID_ORDEN_DE_DESPACHO        46 non-null     object 
 2   ID_DE_ACCION_DE_ORDEN       46 non-null     object 
 3   NOMBRE_DEL_CLIENTE          46 non-null     object 
 4   DOCUMENTO_CLIENTE_TIPO      46 non-null     object 
 5   DOCUMENTO_CLIENTE           46 non-null     object 
 6   DOCUMENTO_CLIENTE_DV        46 non-null     object 
 7   USUARIO_CREACION            46 non-null     object 
 8   TIPO_DE_REQUERIMIENTO       46 non-null     object 
 9   ESTADO                      46 non-null     object 
 10  FECHA_HORA_DEL_ESTADO       46 non-null     object 
 11  COMENTARIO_FLAG_ERROR       46 non-null     object 
 12  PRODUCTO_1                  46 non-null     int64  
 13  PRODUCTO_2                  46 non-nu

In [29]:
  df.head(2)

,ID_ORDEN,ID_ORDEN_DE_DESPACHO,ID_DE_ACCION_DE_ORDEN,NOMBRE_DEL_CLIENTE,DOCUMENTO_CLIENTE_TIPO,DOCUMENTO_CLIENTE,DOCUMENTO_CLIENTE_DV,USUARIO_CREACION,TIPO_DE_REQUERIMIENTO,ESTADO,...,FECHA_ESTADO_AGENDADO,FECHA_ALTA,MONTH_FINALIZATION,SERIE,MSISDN,NUMERO_DE_GUIA_DE_DESPACHO,ID_ASIGNACION_OTR,TIPO_CONFIRMACION,CODIGO_CLIENTE,ESTADO_ANTERIOR
0,eenapsr,entcssrap,entcssray,Cliente_1,RUT,scsptocc,a,usuario_1,Tipo_1,Estado_12,...,"05/12/22 10:40:44,000000000",NaN,NaN,NaN,NaN,NaN,eenapsr,NaN,oeorrepts.c,Estado_4
1,eenapsr,entcssrap,entcssray,Cliente_1,RUT,scsptocc,a,usuario_1,Tipo_1,Estado_12,...,"05/12/22 10:40:44,000000000",NaN,NaN,NaN,NaN,NaN,eenapsr,NaN,oeorrepts.c,Estado_4


Para comenzar con la transformación de los datos, uniremos las columnas "DOCUMENTO_CLIENTE" y "DOCUMENTO_CLIENTE_DV" para crear una nueva columna llamada "RUT". Actualmente se encuentran de la siguiente forma:

In [30]:
  print(df[["DOCUMENTO_CLIENTE","DOCUMENTO_CLIENTE_DV"]].head(2))

  DOCUMENTO_CLIENTE DOCUMENTO_CLIENTE_DV
0          scsptocc                    a
1          scsptocc                    a


Aplicaremos la transformación

In [31]:
  df["RUT"] = df[["DOCUMENTO_CLIENTE","DOCUMENTO_CLIENTE_DV"]].apply("-".join, axis=1)
  print(df["RUT"].head(2))

0    scsptocc-a
1    scsptocc-a
Name: RUT, dtype: object


Ahora, eliminaremos las columnas que no utilizaremos.

In [32]:
  df = df.drop(["ID_ORDEN","ID_DE_ACCION_DE_ORDEN","DOCUMENTO_CLIENTE_TIPO","DOCUMENTO_CLIENTE","DOCUMENTO_CLIENTE_DV","CANAL","FECHA_ALTA","MONTH_FINALIZATION","SERIE","MSISDN","NUMERO_DE_GUIA_DE_DESPACHO","ID_ASIGNACION_OTR"],axis=1)

Verificaremos que se hayan eliminado las columnas

In [33]:
  print(df.columns)

Index(['ID_ORDEN_DE_DESPACHO', 'NOMBRE_DEL_CLIENTE', 'USUARIO_CREACION',
       'TIPO_DE_REQUERIMIENTO', 'ESTADO', 'FECHA_HORA_DEL_ESTADO',
       'COMENTARIO_FLAG_ERROR', 'PRODUCTO_1', 'PRODUCTO_2',
       'SISTEMA_DE_ORIGEN', 'FECHA_ESTADO_AGENDADO', 'TIPO_CONFIRMACION',
       'CODIGO_CLIENTE', 'ESTADO_ANTERIOR', 'RUT'],
      dtype='object')


Guardaremos el archivo transformado, en un nuevo archivo, en este caso csv para mejorar el rendimiento de las próximas transformaciones. No se sobreescribirá el archivo original para evitar perder información valiosa.

In [34]:
  df.to_csv('/content/drive/My Drive/python_projects/CSV/' + i[:12] + '_new.csv')

Ahora, consolidaremos todos los archivos en uno solo, con toda la información, con un pd.concat

In [35]:
consolidado_anual = os.path.join("/content/drive/My Drive/python_projects/CSV/","*.csv")
list_files_2 = glob.glob(consolidado_anual)
df = pd.concat(map(pd.read_csv, list_files_2), ignore_index=True)
print(df.info())
df.to_csv("/content/drive/My Drive/python_projects/CSV/Consolidado.csv",sep=",",encoding="utf-8")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             9152 non-null   int64  
 1   ID_ORDEN_DE_DESPACHO   9152 non-null   object 
 2   NOMBRE_DEL_CLIENTE     9152 non-null   object 
 3   USUARIO_CREACION       9152 non-null   object 
 4   TIPO_DE_REQUERIMIENTO  9152 non-null   object 
 5   ESTADO                 9152 non-null   object 
 6   FECHA_HORA_DEL_ESTADO  9152 non-null   object 
 7   COMENTARIO_FLAG_ERROR  9128 non-null   object 
 8   PRODUCTO_1             9152 non-null   int64  
 9   PRODUCTO_2             9152 non-null   int64  
 10  SISTEMA_DE_ORIGEN      9152 non-null   object 
 11  FECHA_ESTADO_AGENDADO  9152 non-null   object 
 12  TIPO_CONFIRMACION      8616 non-null   float64
 13  CODIGO_CLIENTE         536 non-null    object 
 14  ESTADO_ANTERIOR        9152 non-null   object 
 15  RUT 

Ahora, importaremos el archivo consolidado a un nuevo dataframe

In [36]:
df_consolidado = pd.read_csv('/content/drive/My Drive/python_projects/CSV/Consolidado.csv')
print(df_consolidado.head(2))

   Unnamed: 0.8  Unnamed: 0 ID_ORDEN_DE_DESPACHO NOMBRE_DEL_CLIENTE  \
0             0           0           mcccoyporc          Cliente_1   
1             1           1           mcccoyporc          Cliente_1   

  USUARIO_CREACION TIPO_DE_REQUERIMIENTO     ESTADO FECHA_HORA_DEL_ESTADO  \
0        usuario_1                Tipo_5  Estado_14   2023-01-23 16:15:16   
1        usuario_1                Tipo_5  Estado_14   2023-01-23 16:15:16   

  COMENTARIO_FLAG_ERROR  PRODUCTO_1  ...  CODIGO_CLIENTE ESTADO_ANTERIOR  \
0            En Gestion           0  ...             NaN        Estado_9   
1            En Gestion           0  ...             NaN        Estado_9   

          RUT  Unnamed: 0.7 Unnamed: 0.6 Unnamed: 0.5 Unnamed: 0.4  \
0  ssntrsrt-K           NaN          NaN          NaN          NaN   
1  ssntrsrt-K           NaN          NaN          NaN          NaN   

   Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  
0           NaN           NaN           NaN  
1           NaN       

Convertiremos la columna FECHA_HORA_DEL_ESTADO al tipo de dato 'datetime', para luego ordenar al dataframe por esta columna, del valor más reciente al más antiguo

In [37]:
df_consolidado['FECHA_HORA_DEL_ESTADO'] = pd.to_datetime(df_consolidado['FECHA_HORA_DEL_ESTADO'])
print(df_consolidado['FECHA_HORA_DEL_ESTADO'].head(2))

0   2023-01-23 16:15:16
1   2023-01-23 16:15:16
Name: FECHA_HORA_DEL_ESTADO, dtype: datetime64[ns]


In [38]:
df_consolidado = df_consolidado.sort_values('FECHA_HORA_DEL_ESTADO', ascending=False)
print(df_consolidado.head(5))

      Unnamed: 0.8  Unnamed: 0 ID_ORDEN_DE_DESPACHO NOMBRE_DEL_CLIENTE  \
1040          1040         531           mcccoysspc          Cliente_6   
3374          3374         531           mcccoysspc          Cliente_6   
7950          7950         531           mcccoysspc          Cliente_6   
9094          9094         531           mcccoysspc          Cliente_6   
1041          1041         532           mcccoysspc          Cliente_6   

     USUARIO_CREACION TIPO_DE_REQUERIMIENTO     ESTADO FECHA_HORA_DEL_ESTADO  \
1040        usuario_5                Tipo_5  Estado_14   2023-02-07 15:41:00   
3374        usuario_5                Tipo_5  Estado_14   2023-02-07 15:41:00   
7950        usuario_5                Tipo_5  Estado_14   2023-02-07 15:41:00   
9094        usuario_5                Tipo_5  Estado_14   2023-02-07 15:41:00   
1041        usuario_5                Tipo_5  Estado_14   2023-02-07 15:41:00   

     COMENTARIO_FLAG_ERROR  PRODUCTO_1  ...  CODIGO_CLIENTE ESTADO_ANTERIO

A partir de esto, se eliminarán los registros que en la columna "ID_ORDEN_DE_DESPACHO", se encuentren duplicadas

In [39]:
df_consolidado = df_consolidado.drop_duplicates("ID_ORDEN_DE_DESPACHO")
print(df_consolidado.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 1040 to 2243
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0.8           36 non-null     int64         
 1   Unnamed: 0             36 non-null     int64         
 2   ID_ORDEN_DE_DESPACHO   36 non-null     object        
 3   NOMBRE_DEL_CLIENTE     36 non-null     object        
 4   USUARIO_CREACION       36 non-null     object        
 5   TIPO_DE_REQUERIMIENTO  36 non-null     object        
 6   ESTADO                 36 non-null     object        
 7   FECHA_HORA_DEL_ESTADO  36 non-null     datetime64[ns]
 8   COMENTARIO_FLAG_ERROR  34 non-null     object        
 9   PRODUCTO_1             36 non-null     int64         
 10  PRODUCTO_2             36 non-null     int64         
 11  SISTEMA_DE_ORIGEN      36 non-null     object        
 12  FECHA_ESTADO_AGENDADO  36 non-null     object        
 13  TI

Guardamos el nuevo dataframe en un archivo excel final. Con esto, terminamos la transformación de los datos de este proyecto.

In [40]:
df_consolidado.to_excel('/content/drive/My Drive/python_projects/CSV/Consolidado_sin_duplicados.xlsx')